# Create network graph and analysis

# 1) Set up libraries and work directories

In [ ]:
# Set up libraries
import os
import time 
import random
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from rich.text import Text
from rich.console import Console
from pyvis.network import Network
with pd.ExcelWriter("myfile.xlsx", engine="xlsxwriter") as writer:

# Work directory
input_directory = "INPUT_DIRECTORY"
variantscape_directory = "VARIANTSCAPE_DIRECTORY"
variantscape_llm_coas_directory = "VARIANTSCAPE_LLM_COAS_DIRECTORY"

# Load the metadata and variant dataset
os.chdir(variantscape_directory)
metadata_mapping = pd.read_csv(os.path.join(variantscape_directory, "metadata_mapping_transposed.csv"), low_memory=False)
variant_analysis_df = pd.read_csv(os.path.join(variantscape_directory, "cleaned_df_v4.csv"), low_memory=False)
print(metadata_mapping.head(5))

os.chdir(variantscape_LLM_coas_directory)
df_consensus = pd.read_csv("final_variant_treatment_consensus.csv")
os.chdir(variantscape_directory)
print("\n\n")
print(df_consensus.head(5))

In [ ]:
##### Explore variant dictionary
# Filter metadata_mapping for entities where Category == 'Variant'
variant_entities = metadata_mapping[metadata_mapping['Category'] == 'Variant']['Entity'].tolist()
print(f"\nTotal unique entities with Category == 'Variant': {len(variant_entities)}")
print("ScoreLabel List for Entity = Variant:\n")
for idx, entity in enumerate(variant_entities, 1):
    print(f"{idx}. {entity}")

In [ ]:
# Cancer dictionary
# Filter metadata_mapping for entities where Category == 'Cancer'
cancer_entities = metadata_mapping[metadata_mapping['Category'] == 'Cancer']['Entity'].tolist()
print(f"\nTotal unique entities with Category == 'Cancer': {len(cancer_entities)}")
print("ScoreLabel List for Entity = Cancer:\n")
for idx, entity in enumerate(cancer_entities, 1):
    print(f"{idx}. {entity}")

In [ ]:
# Treatment dictionary
# Filter for Treatment entities
treatment_entities = metadata_mapping[metadata_mapping['Category'] == 'Treatment']['Entity'].tolist()
print(f"\n\033[1mTotal unique entities with Category == 'Treatment': {len(treatment_entities)}\033[0m")
print("\n\033[1mScoreLabel List for Entity = Treatment:\033[0m")
for idx, entity in enumerate(treatment_entities, 1):
    print(f"{idx}. {entity}")

# ========================================================

# 2) Network graph analysis

## Create network graph

In [ ]:
# Clean variant_analysis_df by removing non-entity columns
# Ingnore non binary columns
entity_columns = [col for col in variant_analysis_df.columns if col not in ['PaperId', 'Study_design', 'Abstract', 'Study_weight', 'PaperTitle']]

# Create network graph
G = nx.Graph()

# Add nodes for each entity based on metadata_mapping dictionary
for col in entity_columns:
    category = metadata_mapping.loc[metadata_mapping['Entity'] == col, 'Category'].values[0]
    G.add_node(col, category=category)

# Add edges based on co-occurrence
for idx, row in tqdm(variant_analysis_df.iterrows(), total=variant_analysis_df.shape[0], desc="Adding edges"):
    present_entities = row[entity_columns] == 1 
    present_columns = present_entities[present_entities].index
    for col1 in present_columns:
        for col2 in present_columns:
            if col1 != col2:
                if G.has_edge(col1, col2):
                    G[col1][col2]['weight'] += 1 
                else:
                    G.add_edge(col1, col2, weight=1)

# Exmplore network analysis
print("Number of nodes:", len(G.nodes))
print("Number of edges:", len(G.edges))
print("Network successfully created!")

# Save the network
nx.write_gml(G, 'network_graph.gml')

In [ ]:
# Verify the network analysis
G = nx.read_gml('network_graph.gml')

# Example query: Find the neighbors of a particular variant of interest
variant_of_interest = "s768i_EGFR"
variant_neighbors = set(G.neighbors(variant_of_interest))

# Find treatments and cancers associated with the variant
treatments = []
cancers = []

for node in variant_neighbors:
    if G.nodes[node]['category'] == 'Treatment':
        treatments.append(node)
    elif G.nodes[node]['category'] == 'Cancer':
        cancers.append(node)
        
print(f"Treatments associated with variant '{variant_of_interest}':")
print(treatments)
print(f"\nCancers associated with variant '{variant_of_interest}':")
print(cancers)

# Find the top 5 most connected nodes (by degree centrality)
centrality = nx.degree_centrality(G)
sorted_centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)
print("\nTop 5 most connected nodes (based on degree centrality):")
for node, score in sorted_centrality[:5]:
    print(f"{node}: {score:.4f}")

## Create weighted network graph based on study design

In [ ]:
# Create weighted network graph based on study design

# Study‑design weights
study_design_weights = {
    'Systematic review study':       1.0,
    'Clinical study':                1.0,
    'Observational/RWE study':       0.9,
    'Case report study':             0.9,
    'In vivo/Animal study':          0.8,
    'In vitro study':                0.7,
    'In silico study':               0.6,
    'Undefined':                     0.1,
    'Other':                         0.1,
}

# Entity columns
non_entity_cols = ['PaperId', 'Study_design', 'Abstract', 'Study_weight', 'PaperTitle']
entity_columns = [c for c in variant_analysis_df.columns if c not in non_entity_cols]

# Initialize two graphs
G   = nx.Graph()  # unweighted (increments by 1)
G_w = nx.Graph()  # weighted by evidence level

# 1) Add nodes to both
for col in entity_columns:
    cat = metadata_mapping.loc[
        metadata_mapping['Entity'] == col, 'Category'
    ].values[0]
    G.add_node(col,   category=cat)
    G_w.add_node(col, category=cat)

# 2) Add edges
for _, row in tqdm(variant_analysis_df.iterrows(), total=len(variant_analysis_df), desc="Building graphs"):
    ents = row[entity_columns][row[entity_columns] == 1].index.tolist()
    design = str(row['Study_design']).strip()
    w      = study_design_weights.get(design, 0.5)

    # iterate unique pairs
    for i, e1 in enumerate(ents):
        for e2 in ents[i+1:]:
            # --- unweighted: +1 per co‑occurrence ---
            if G.has_edge(e1, e2):
                G[e1][e2]['weight'] += 1
            else:
                G.add_edge(e1, e2, weight=1)

            # --- weighted: +w per co‑occurrence ---
            if G_w.has_edge(e1, e2):
                G_w[e1][e2]['weight'] += w
            else:
                G_w.add_edge(e1, e2, weight=w)

# 3) Compare basic metrics
print("=== Unweighted graph")
print(" Nodes:", G.number_of_nodes(), "Edges:", G.number_of_edges())
deg_unw = nx.degree_centrality(G)
top_unw = sorted(deg_unw.items(), key=lambda x: x[1], reverse=True)[:5]
print(" Top 5 nodes by degree centrality:", top_unw)
print("\n=== Weighted graph")
print(" Nodes:", G_w.number_of_nodes(), "Edges:", G_w.number_of_edges())
deg_w = nx.degree_centrality(G_w)
top_w = sorted(deg_w.items(), key=lambda x: x[1], reverse=True)[:5]
print(" Top 5 nodes by degree centrality:", top_w)

# Compare edge‐weight distributions
weights_unw = np.array([d['weight'] for _, _, d in G.edges(data=True)])
weights_w   = np.array([d['weight'] for _, _, d in G_w.edges(data=True)])

print("Edge weights (unweighted): mean=%.2f, std=%.2f" % (weights_unw.mean(), weights_unw.std()))
print("Edge weights (weighted):   mean=%.2f, std=%.2f" % (weights_w.mean(),   weights_w.std()))
print("Number of edges re‑weighted:", np.sum(weights_w != weights_unw))

# Correlate the two weight vectors
corr = np.corrcoef(weights_unw, weights_w)[0,1]
print("Pearson corr between unweighted/weighted edge‐weights: %.3f" % corr)

# Compare degree centrality shifts per node
dc_unw = nx.degree_centrality(G)
dc_w   = nx.degree_centrality(G_w)
delta  = {n: dc_w[n] - dc_unw[n] for n in G.nodes()}
nx.write_gml(G_w, 'network_graph_weighted.gml')

# =================================================

# 3) Query the network to find associations

In [ ]:
# Load datasets needed for network query
# UPDATE Load (or reload) the weighted graph into G_w, then alias it to G for downstream code!!
try:
    G_w
    print("Weighted graph already loaded. Proceeding with analysis.")
except NameError:
    print("Loading the weighted network graph...")
    path = os.path.join(variantscape_directory, "network_graph_weighted.gml")
    G_w = nx.read_gml(path)
# Use G as the graph variable everywhere else
G = G_w.copy()

try:
    df_consensus
    print("df_consensus already loaded. Proceeding with analysis.")
except NameError:
    print("Loading the consensus file...")
    df_consensus_path = variantscape_LLM_coas_directory + '/final_variant_treatment_consensus.csv'
    df_consensus = pd.read_csv(df_consensus_path)
    
# Define variant and cancer of interest (with aliasing)
user_input_cancer = "NSCLC"
#variant_of_interest = "v600e_BRAF"

############## Variants of interest for publication ##############
#variant_of_interest = 'l858r_EGFR' #as durggable usecase
#variant_of_interest = 't790m_EGFR' #as resistant usecase

######################## Rare variants #############################
#variant_of_interest = 'g469v_BRAF'
#variant_of_interest = 'g719x_EGFR' # does not exist in network
#variant_of_interest = 's768i_EGFR'
variant_of_interest = 'l861q_EGFR'
#variant_of_interest = 'l747p_EGFR' # no information in the network


# Define alias mapping for cancer names
cancer_alias_map = {
    "nsclc": "lung cancer",
    "non-small cell lung cancer": "lung cancer",
    "tnbc": "breast cancer",
    "her2+ breast cancer": "breast cancer"
}

# Normalize input
clean_input = user_input_cancer.strip().lower()

# Use alias if it exists, otherwise just use the same name
cancer_of_interest = cancer_alias_map.get(clean_input, clean_input)
display_cancer_name = user_input_cancer

print(f"\n\n\033[1mCancer of interest set to:\033[0m {display_cancer_name} (cancer type:'{cancer_of_interest}')")
print(f"\033[1mVariant of interest set to:\033[0m {variant_of_interest}")

In [ ]:
#### Updated weighted network graph with automated threshold, based on qualitative analysis

# Adjustable thresholds
TREATMENT_THRESHOLD_PERCENTILE = 80    # highlight top X% of treatment weights
TREATMENT_MIN_HIGHLIGHT        = 300   # and require ≥X total weight
CANCER_THRESHOLD_PERCENTILE    = 80    # highlight top X% of cancer–variant weights
CANCER_MIN_HIGHLIGHT           = 80    # and require ≥X total weight

# Prepare consensus lookup
df_consensus["Variant_Treatment_Pair"] = (
    df_consensus["Variant_Treatment_Pair"]
    .str.strip()
    .str.lower()
)
consensus_dict = dict(
    zip(df_consensus["Variant_Treatment_Pair"], df_consensus["Resolved_Prediction"])
)

excluded_treatments = {
    'chemotherapy', 'tyrosine kinase inhibitor', 'radiotherapy', 'hormone therapy',
    'adjuvant chemotherapy', 'immunotherapy', 'immune checkpoint inhibitor',
    'mrna vaccine', 'mtor inhibitor', 'radiation ionizing radiotherapy'
}

# Step 1: Cancer‐only treatments
canc_nei = set(G.neighbors(cancer_of_interest))
treatments = [
    n for n in canc_nei
    if G.nodes[n]['category']=='Treatment'
    and n.lower() not in excluded_treatments
]
t_weights = {t: G[cancer_of_interest][t]['weight'] for t in treatments}
top_cancer_treats = sorted(t_weights.items(), key=lambda x: x[1], reverse=True)[:6]
c_w = list(t_weights.values())
treat_pct = np.percentile(c_w, TREATMENT_THRESHOLD_PERCENTILE) if c_w else 0


# Step 2: Variant + cancer associations
sensitive, resistant = [], []
for t in treatments:
    try:
        w = G[cancer_of_interest][t]['weight'] + G[variant_of_interest][t]['weight']
        pred = consensus_dict.get(f"{variant_of_interest} + {t}".lower())
        if pred == "Sensitive":
            sensitive.append((t, w))
        elif pred == "Resistant":
            resistant.append((t, w))
    except KeyError:
        continue

top_sens = sorted(sensitive, key=lambda x: x[1], reverse=True)[:6]
top_res  = sorted(resistant, key=lambda x: x[1], reverse=True)[:6]
sens_w = [w for _, w in sensitive]
res_w  = [w for _, w in resistant]
sens_pct = np.percentile(sens_w, TREATMENT_THRESHOLD_PERCENTILE) if sens_w else 0
res_pct  = np.percentile(res_w,   TREATMENT_THRESHOLD_PERCENTILE) if res_w else 0

print(f"\n\033[1mSensitive treatments for variant '{variant_of_interest}' "
      f"(≥{TREATMENT_THRESHOLD_PERCENTILE}th pct & ≥{TREATMENT_MIN_HIGHLIGHT}):\033[0m")
for t, w in top_sens:
    if w >= sens_pct and w >= TREATMENT_MIN_HIGHLIGHT:
        print(f"\033[1;32m{t}: {w:.0f}\033[0m")
    else:
        print(f"\033[2;37m{t}: {w:.0f}\033[0m")

print(f"\n\033[1mResistant treatments for variant '{variant_of_interest}' "
      f"(≥{TREATMENT_THRESHOLD_PERCENTILE}th pct & ≥{TREATMENT_MIN_HIGHLIGHT}):\033[0m")
for t, w in top_res:
    if w >= res_pct and w >= TREATMENT_MIN_HIGHLIGHT:
        print(f"\033[1;31m{t}: {w:.0f}\033[0m")
    else:
        print(f"\033[2;37m{t}: {w:.0f}\033[0m")

# Step 3: Other cancers for variant
var_nei = set(G.neighbors(variant_of_interest))
var_cancers = [
    n for n in var_nei
    if G.nodes[n]['category']=='Cancer'
    and n != cancer_of_interest
]
vc_weights = {}
for c in var_cancers:
    w_v = G[variant_of_interest][c]['weight']
    w_c = G[cancer_of_interest][c]['weight'] if G.has_edge(cancer_of_interest, c) else 0
    vc_weights[c] = w_v + w_c

top_var_c = sorted(vc_weights.items(), key=lambda x: x[1], reverse=True)[:6]
vc_w = list(vc_weights.values())
cancer_pct = np.percentile(vc_w, CANCER_THRESHOLD_PERCENTILE) if vc_w else 0

print(f"\n\033[1mOther cancers for variant '{variant_of_interest}' "
      f"(≥{CANCER_THRESHOLD_PERCENTILE}th pct & ≥{CANCER_MIN_HIGHLIGHT}):\033[0m")
for c, w in top_var_c:
    if w >= cancer_pct and w >= CANCER_MIN_HIGHLIGHT:
        print(f"\033[1;34m{c}: {w:.0f}\033[0m")
    else:
        print(f"\033[2;37m{c}: {w:.0f}\033[0m")

# Assemble & save
results = []
for t, w in top_cancer_treats:
    results.append({
        "Cancer": display_cancer_name, "Variant": None,
        "Treatment": t, "Association_Type": "Cancer-Only",
        "Prediction": "NA", "Combined_Weight": w
    })
for t, w in top_sens:
    results.append({
        "Cancer": display_cancer_name, "Variant": variant_of_interest,
        "Treatment": t, "Association_Type": "Variant-Cancer",
        "Prediction": "Sensitive", "Combined_Weight": w
    })
for t, w in top_res:
    results.append({
        "Cancer": display_cancer_name, "Variant": variant_of_interest,
        "Treatment": t, "Association_Type": "Variant-Cancer",
        "Prediction": "Resistant", "Combined_Weight": w
    })
for c, w in top_var_c:
    results.append({
        "Cancer": c, "Variant": variant_of_interest,
        "Treatment": None, "Association_Type": "Cross-Cancer",
        "Prediction": "NA", "Combined_Weight": w
    })

df_out = pd.DataFrame(results)
safe_c = display_cancer_name.replace(" ", "_").lower()
safe_v = variant_of_interest.replace(" ", "_").lower()

csv_fname   = f"network_results_{safe_c}_{safe_v}.csv"
excel_fname = f"network_results_{safe_c}_{safe_v}.xlsx"

df_out.to_csv(csv_fname, index=False)
print(f"\nCSV saved to: {csv_fname}")

with pd.ExcelWriter(excel_fname, engine='xlsxwriter') as w:
    df_out.to_excel(w, sheet_name='Results', index=False)
print(f"Excel saved to: {excel_fname}")

# =================================================

# 4) Create network figure

In [ ]:
# Create figure
try:
    G
    print("Graph already loaded. Proceeding to visualization.")
except NameError:
    print("Loading the network graph...")
    gml_file_path = variantscape_directory + '/network_graph.gml'
    for _ in tqdm(range(100), desc="Loading Graph", ncols=100, ascii=True):
        time.sleep(0.01)
    G = nx.read_gml(gml_file_path)

# Select nodes from each category to ensure each category is included
variant_nodes = [node for node, data in G.nodes(data=True) if data.get('category') == 'Variant']
treatment_nodes = [node for node, data in G.nodes(data=True) if data.get('category') == 'Treatment']
cancer_nodes = [node for node, data in G.nodes(data=True) if data.get('category') == 'Cancer']

# Downsample the graph and ensure representation of each category
# Sample all nodes if there are fewer than 1000 in a category!!!
subset_variant_nodes = random.sample(variant_nodes, min(1000, len(variant_nodes)))
subset_treatment_nodes = random.sample(treatment_nodes, min(1000, len(treatment_nodes)))
subset_cancer_nodes = random.sample(cancer_nodes, min(1000, len(cancer_nodes)))
subset_nodes = subset_variant_nodes + subset_treatment_nodes + subset_cancer_nodes

# Create subgraph with selected nodes
subset_edges = [(u, v) for u, v in G.edges() if u in subset_nodes and v in subset_nodes]
G_sub = G.subgraph(subset_nodes).copy()
G_sub.add_edges_from(subset_edges)

# Initialize the PyVis network
net = Network(notebook=True, height="800px", width="100%", directed=False, cdn_resources='in_line')
category_colors = {
    'Variant': 'lightblue',  
    'Treatment': 'darkgreen',  
    'Cancer': 'red'   
}

print("Adding graph to visualization...")
for _ in tqdm(range(100), desc="Adding Graph to PyVis", ncols=100, ascii=True):
    time.sleep(0.01)
net.from_nx(G_sub)
print("Customizing nodes and edges...")
for node in tqdm(net.nodes, desc="Customizing Nodes", ncols=100, ascii=True):
    node_id = node['id']
    category = G_sub.nodes[node_id].get('category', 'variant') 
    node_color = category_colors.get(category, 'gray') 
    node['color'] = node_color
    node['title'] = f"Node Info: {node_id}"
    node['size'] = 20 
for edge in net.edges:
    edge['color'] = 'whitesmoke' 
    edge['width'] = 0.3  

print("Applying force-directed layout...")
net.force_atlas_2based()

# Save the network as an HTML file (open separately in new window!!!)
output_file = "downsampled_network_visualization_with_custom_colors_and_nodes.html"
print("Saving network as HTML file...")
for _ in tqdm(range(100), desc="Saving Network", ncols=100, ascii=True):
    time.sleep(0.01)
net.save_graph(output_file)
print(f"Network visualization saved as {output_file}. Open it in a browser to explore.")